## `equal() ` vs `==` operator
---

> In reference types(not primitive types), `==` checks if both reference point to the same memory location whereas `.equals()` evaluates to the comparison of contents in the referenced objects. 

```java
public class Test {
	public static void main(String[] args)
	{
		String s1 = "HELLO";
		String s2 = "HELLO";
		String s3 = new String("HELLO");

		System.out.println(s1 == s2); // true
		System.out.println(s1 == s3); // false
		System.out.println(s1.equals(s2)); // true
		System.out.println(s1.equals(s3)); // true
	}
}
```

## 为什么每个对象都有`hashCode()`方法
---

Java `equals()` and `hashCode()` methods are present in `Object` class. So every java class gets the default implementation of `equals()` and `hashCode()`.

- If `o1.equals(o2)`, then `o1.hashCode() == o2.hashCode()` should always be `true`.
- If `o1.hashCode() == o2.hashCode` is `true`, it doesn’t mean that `o1.equals(o2)` will be `true`.

`hashCode()`用于返回对象的hash值(`int`类型)，主要用在collection的HashSet和Map中, 用于快速判断每个集合的元素的位置. 另外注意哦, HashMap会用到两个方法: `hash()`和`hashCode()`, 其中`hashCode()`很显然就是那个所谓的hash function, 而`hash()`是干啥的呢?

> 答案: `hash()`是为了增加随机性，让数据元素**更加**均衡的分布，减少碰撞, 就是通过对象的`hashCode()`获得它的hash值后, 再用`hash()`处理一下, 这样就是加强版. 

看看HashMap的`put()`源码:

```java
public V put(K key, V value) {
    return putVal(hash(key), key, value, false, true);
}
```

`hash()`的源码:

```java
static final int hash(Object key) {
    int h;
    return (key == null) ? 0 : (h = key.hashCode()) ^ (h >>> 16);
}
```

可以看到, 获取到hash值(`h = key.hashCode()`)之后还要与hash值的高位(前16位)做异或运算(`^ (h >>> 16)`), 之后获得的是加强版hash值.

那看看`putVal(hash(key), key, value, false, true)`是怎么用的这个加强版hash值呢?

```java
    final V putVal(int hash, K key, V value, boolean onlyIfAbsent,
                   boolean evict) {
        Node<K,V>[] tab; Node<K,V> p; int n, i;
        if ((tab = table) == null || (n = tab.length) == 0)
            n = (tab = resize()).length;
        if ((p = tab[i = (n - 1) & hash]) == null)
            tab[i] = newNode(hash, key, value, null);
        else {
            ...
            ...
        }
        ...
        if (++size > threshold)
            resize();
        ...
    }
```

可以看到`p = tab[i = (n - 1) & hash]`, 显然`(n - 1) & hash`是计算index. 就是你向HashMap插入一个元素, 这个元素在hash table的位置. 